In [12]:
import os
import sys
import pandas as pd
import numpy as np
from IPython.display import display
from enum import auto, Enum
import getpass

In [15]:
class Wordler():
#     Static stuff (i.e. instance independent)
    @staticmethod
    def get_english_words_path():
        this_dir_path = os.path.dirname(os.path.realpath('__file__'))
#         print (this_dir_path) 
        # These are all the english words
        english_words_path = os.path.abspath(os.path.join(this_dir_path, 'english-words'))
        return english_words_path

    @staticmethod
    def reset_word_list(word_length: int = 5):
        english_words_path = Wordler.get_english_words_path()
        words_alpha_list = pd.read_csv(os.path.join(english_words_path, 'words_alpha.txt'), header=None, names=['Words'], dtype='str')
    #     display(words_alpha_list)
        mask = (words_alpha_list.Words.str.len() == word_length)
        wordle_valid_list = words_alpha_list[mask]
        wordle_valid_list.to_csv(os.path.join(english_words_path, 'valid_wordles.txt'), header=None, index=None)

    @staticmethod
    def get_word_list():
        english_words_path = Wordler.get_english_words_path()
        wordle_valid_list = pd.read_csv(os.path.join(english_words_path, 'valid_wordles.txt'), header=None, names=['Words'], dtype='str').Words.values
#         display(wordle_valid_list)
        return wordle_valid_list

# Initialise
    def __init__(self, word_length: int = 5):
        self.word_length = 5
        if not(word_length is None):
            if word_length != 5:
                Wordler.reset_word_list(word_length)
                self.word_length = word_length                
        self.valid_words = Wordler.get_word_list()
    def is_valid_wordle(self, word:str):
        word = str.lower(word)
        is_valid_wordle = word in self.valid_words
        return is_valid_wordle

    class Wordle_Status(Enum):
        PLAYING = auto()
        WON = auto()
        LOST = auto()
        
    class Guess_Result(Enum):
        NotInWord = auto()
        InCorrectPlace = auto()
        CorrectPlace = auto()
        
    
# Gaming stuff
class Game():
    class Guess_Result():
        def __init__(self, 
                     msg = '', 
                     status: Wordler.Wordle_Status = Wordler.Wordle_Status.PLAYING, 
                     tries_left: int = 6, 
                     results: dict = dict(),
                     guesses: list = []
                    ):
            self.msg = msg
            self.results = results
            self.status = status
            self.tries_left = tries_left

    def __init__(self,word_to_guess: str):
        self.engine = Wordler() # Initialises and caches valid words
        word_to_guess = str.lower(word_to_guess)
        is_valid_wordle = self.engine.is_valid_wordle(word_to_guess)
        if is_valid_wordle:
            print(f'Good start buddy. That word is indeed a valid Wordle. ;-)')
        else:
             raise Exception(f'Come on!!!! Jeez! "{word_to_guess}" is not a valid Wordle.')   
        self.word_to_guess = word_to_guess
        self.max_tries = 6
        self.tries_left = self.max_tries
        self.status = Wordler.Wordle_Status.PLAYING
        self.guesses = []
        self.results_per_guess = []

    def tries_left(self):
        return self.tries_left

    def status(self):
        return self.status

    def guesses_msg(self):
        tries_taken = self.max_tries - self.tries_left
        msg = ''
        msg += 'Your guesses have been: ' + ','.join(self.guesses) + '\n'
        msg += f'You have used up {tries_taken}/{self.max_tries} tries.\n'
        return msg

    def won_result(self, results:dict = dict()):
        tries_taken = self.max_tries - self.tries_left
        guesses = ','.join(self.guesses)
        msg = f'You already won in {tries_taken}/{self.max_tries} tries.\n'
        msg += 'Congratulations.' + self.guesses_msg() + '\n'
        msg += f'Wordle was {self.word_to_guess}.'
        result = Game.Guess_Result(
            msg = msg, 
            status = self.status,
            tries_left = self.tries_left,
            results = results,
            guesses = self.guesses
        )
        print('__________________________________________________________________________')
        print(result.msg)
        print('__________________________________________________________________________')
        return result

    def lost_result(self, results:dict = dict()):
        tries_taken = self.max_tries - self.tries_left
        guesses = ','.join(self.guesses)
        msg = f'You already lost in {tries_taken}.\n'
        msg += 'Keep playing.' + self.guesses_msg() + '\n'
        msg += f'Wordle was {self.word_to_guess}.'
        result = Game.Guess_Result(
            msg = msg, 
            status = self.status,
            tries_left = self.tries_left,
            results = results,
            guesses = self.guesses
        )
        print('__________________________________________________________________________')
        print(result.msg)
        print('__________________________________________________________________________')
        return result

    def invalid_guess_result(self, msg: str = ''):
        tries_taken = self.max_tries - self.tries_left
        guesses = ','.join(self.guesses)
        msg = f'Invalid Entry because: {msg}' + '\n'
        msg += 'Keep playing.' + self.guesses_msg()          
        result = Game.Guess_Result(
            msg = msg, 
            status = self.status,
            tries_left = self.tries_left,
            guesses = self.guesses
        )
        print('__________________________________________________________________________')
        print(result.msg)
        print('__________________________________________________________________________')
        return result


    def playing_result(self, results:dict):
        tries_taken = self.max_tries - self.tries_left
        msg = 'Keep playing.' + self.guesses_msg()
        result = Game.Guess_Result(
            msg = msg, 
            status = self.status,
            tries_left = self.tries_left,
            results = results,
            guesses = self.guesses
        )
        print('__________________________________________________________________________')
        print(result.msg)
        print('__________________________________________________________________________')
        return result

    def guess(self, word:str):
        word = str.lower(word)
        if (self.status == Wordler.Wordle_Status.WON):
            return self.won_result()
        if (self.status == Wordler.Wordle_Status.LOST):
            return self.lost_result()
        if (word in self.guesses):
            return self.invalid_guess_result(f'{word} is in previous guesses.')
        is_valid_wordle = self.engine.is_valid_wordle(word)
        if not (is_valid_wordle):
            return self.invalid_guess_result(f'{word} is not in list of words. Please try again.')
        else:
            self.tries_left -= 1
            results = {}
            correct_guesses = 0
            self.guesses.append(word)
            for i in range(self.engine.word_length):
                if (word[i] == self.word_to_guess[i]):
                    this_result = Wordler.Guess_Result.CorrectPlace
                    correct_guesses += 1
                elif (word[i] in self.word_to_guess): 
                    this_result = Wordler.Guess_Result.InCorrectPlace
                else:
                    this_result = Wordler.Guess_Result.NotInWord
                results[word[i]] = this_result
                print(f'{word[i]} : {this_result}')


            self.results_per_guess.append(results)
            if (correct_guesses == self.engine.word_length):
                self.status = Wordler.Wordle_Status.WON
                return self.won_result(results=results)
            else:
                if(self.tries_left == 0):
                    self.status = Wordler.Wordle_Status.LOST
                    return self.lost_result(results=results)
                else:
                    return self.playing_result(results=results)
    def play_manual(self):
        def is_number(guess):
            try:
                # Convert it into integer
                val = int(guess)
                return True
            except ValueError:
                try:
                    # Convert it into float
                    val = float(guess)
                    return True
                except ValueError:
                    return False

        while self.status == Wordler.Wordle_Status.PLAYING:
            this_guess = input('Enter your guess (enter number to quit the game): ')
            if is_number(this_guess):
                print('Sorry to see you go. Lets play another time.')
                break
            self.guess(this_guess)





In [16]:
def create_wordler():
    msg = 'Welcome WORDLE host. What is the word that people need to guess? '
    word_to_guess = getpass.getpass(msg)
    game = Game(word_to_guess)
    return game

In [19]:
game = create_wordler()

Welcome WORDLE host. What is the word that people need to guess?  ·····


Good start buddy. That word is indeed a valid Wordle. ;-)


In [20]:
game.play_manual()

Enter your guess (enter number to quit the game):  whats


w : Guess_Result.CorrectPlace
h : Guess_Result.CorrectPlace
a : Guess_Result.InCorrectPlace
t : Guess_Result.InCorrectPlace
s : Guess_Result.NotInWord
__________________________________________________________________________
Keep playing.Your guesses have been: whats
You have used up 1/6 tries.

__________________________________________________________________________


Enter your guess (enter number to quit the game):  butts


b : Guess_Result.NotInWord
u : Guess_Result.NotInWord
t : Guess_Result.InCorrectPlace
t : Guess_Result.InCorrectPlace
s : Guess_Result.NotInWord
__________________________________________________________________________
Keep playing.Your guesses have been: whats,butts
You have used up 2/6 tries.

__________________________________________________________________________


Enter your guess (enter number to quit the game):  abrad


...

c : Guess_Result.NotInWord
l : Guess_Result.NotInWord
e : Guess_Result.CorrectPlace
a : Guess_Result.CorrectPlace
t : Guess_Result.CorrectPlace
__________________________________________________________________________
Keep playing.Your guesses have been: whats,butts,brood,cleat
You have used up 4/6 tries.

__________________________________________________________________________


Enter your guess (enter number to quit the game):  clash


c : Guess_Result.NotInWord
l : Guess_Result.NotInWord
a : Guess_Result.InCorrectPlace
s : Guess_Result.NotInWord
h : Guess_Result.InCorrectPlace
__________________________________________________________________________
Keep playing.Your guesses have been: whats,butts,brood,cleat,clash
You have used up 5/6 tries.

__________________________________________________________________________


Enter your guess (enter number to quit the game):  burns


b : Guess_Result.NotInWord
u : Guess_Result.NotInWord
r : Guess_Result.NotInWord
n : Guess_Result.NotInWord
s : Guess_Result.NotInWord
__________________________________________________________________________
You already lost in 6.
Keep playing.Your guesses have been: whats,butts,brood,cleat,clash,burns
You have used up 6/6 tries.

Wordle was wheat.
__________________________________________________________________________
